In [1]:
import os
import random
import shutil
import time
import warnings

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim

import torch.utils.data
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

In [2]:
from torch.cuda.amp import GradScaler
from torch.cuda.amp import autocast

In [3]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [4]:
!pip install wandb

In [5]:
import wandb
wandb.login()

wandb: Currently logged in as: seankoval. Use `wandb login --relogin` to force relogin


True

In [6]:
wandb.init(project="w251_hw9", entity="seankoval")

In [7]:
#721fdeb49f2aeea9c5eddef6eea98f3a1ce586d0

In [8]:
SEED=2

random.seed(SEED)
torch.manual_seed(SEED)
cudnn.deterministic = True

In [9]:
torch.cuda.device_count()

1

In [10]:
START_EPOCH = 0

In [11]:
ARCH = 'resnet18'
EPOCHS = 2
LR = 0.1
MOMENTUM = 0.9
WEIGHT_DECAY = 1e-4
PRINT_FREQ = 10
TRAIN_BATCH=500
VAL_BATCH=500
WORKERS=2
# "/home/ubuntu/data/train"
TRAINDIR="/data/train"
# /home/ubuntu/data/val
VALDIR="/data/val"

In [12]:
if not torch.cuda.is_available():
    print('GPU not detected.. did you pass through your GPU?')

In [13]:
wandb.init(config={"epochs": EPOCHS, "batch_size": TRAIN_BATCH, "momentum": MOMENTUM, "WEIGHT_DECAY": WEIGHT_DECAY, "arch": ARCH})

In [14]:
GPU = 0

In [15]:
torch.cuda.set_device(GPU)

In [16]:
global_step = 0

In [17]:
def train(train_loader, model, criterion, optimizer, epoch):
    global global_step    
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time, losses, top1, top5],
        prefix="Epoch: [{}]".format(epoch))

    # Grad Scaler
    scaler = GradScaler()
    # switch to train mode
    model.train()

    end = time.time()
    for i, (images, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)
        optimizer.zero_grad()

        if GPU is not None:
            images = images.cuda(GPU, non_blocking=True)
        if torch.cuda.is_available():
            target = target.cuda(GPU, non_blocking=True)

        # compute output
        with autocast():
          output = model(images)
          loss = criterion(output, target)

        # measure accuracy and record loss
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        losses.update(loss.item(), images.size(0))
        top1.update(acc1[0], images.size(0))
        top5.update(acc5[0], images.size(0))

        # compute gradient and do SGD step
        # optimizer.zero_grad()
        # loss.backward()
        # optimizer.step()
        
        # use the scaler
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
        writer.add_scalar("Loss/train", loss, global_step = global_step)
        writer.add_scalar("acc1/train", top1.avg, global_step = global_step)
        writer.add_scalar("acc5/train", top5.avg, global_step = global_step)
        
        wandb.log({"Loss/train": loss, 'acc1/train': top1.avg, 'acc5/train': top5.avg})
        
        global_step = global_step + 1

        if i % PRINT_FREQ == 0:
            progress.display(i)

In [18]:
def validate(val_loader, model, criterion):
    global global_step    
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, top1, top5],
        prefix='Test: ')

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (images, target) in enumerate(val_loader):
            if GPU is not None:
                images = images.cuda(GPU, non_blocking=True)
            if torch.cuda.is_available():
                target = target.cuda(GPU, non_blocking=True)

            # compute output
            output = model(images)
            loss = criterion(output, target)

            # measure accuracy and record loss
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))
            top5.update(acc5[0], images.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % PRINT_FREQ == 0:
                progress.display(i)

        # TODO: this should also be done with the ProgressMeter
        print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))
    writer.add_scalar("Loss/val", losses.avg, global_step = global_step)
    writer.add_scalar("acc1/val", top1.avg, global_step = global_step)
    writer.add_scalar("acc5/val", top5.avg, global_step = global_step)    
    
    wandb.log({"Loss/val": losses.avg, 'acc1/val': top1.avg, 'acc5/val': top5.avg})
    
    global_step = global_step + 1

    return top1.avg

In [19]:
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')

In [20]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

In [21]:
class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

In [22]:
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = LR * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [23]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [24]:
cudnn.benchmark = True

In [25]:
import torch.distributed as dist
import sys

In [26]:
WORLD_SIZE = 2
BACKEND = 'nccl'
# set the URL (tcp port for backend)
URL = 'tcp://54.173.7.235:1883'

RANK = 1

dist.init_process_group(backend = BACKEND, init_method=URL,rank=RANK, world_size=WORLD_SIZE)

In [27]:
imagenet_mean_RGB = [0.47889522, 0.47227842, 0.43047404]
imagenet_std_RGB = [0.229, 0.224, 0.225]

In [28]:
normalize = transforms.Normalize(mean=imagenet_mean_RGB, std=imagenet_std_RGB)

In [29]:
IMG_SIZE = 224
NUM_CLASSES = 1000

In [30]:
model = models.__dict__[ARCH]()

In [31]:
inf = model.fc.in_features

In [32]:
model.fc = nn.Linear(inf, NUM_CLASSES)

In [33]:
torch.distributed.is_initialized()

True

In [34]:
torch.distributed.is_mpi_available()

True

In [35]:
torch.distributed.is_nccl_available()

True

In [36]:
#torch.distributed.is_torchelastic_launched()
#https://pytorch.org/docs/stable/distributed.html#https://oboiko.medium.com/distributed-training-with-pytorch-d1fa5f57b40

In [37]:
# sudo lsof -i -P -n
# use to debug what ports are being accessed by the python3 code for both the worker and master nodes. (It should be set by MASTER_NODE)

In [38]:
#https://leimao.github.io/blog/PyTorch-Distributed-Training/

In [39]:
model.cuda(GPU)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [40]:
model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[GPU])

ip-172-31-28-46:536:536 [0] NCCL INFO Bootstrap : Using ens5:172.31.28.46<0>
ip-172-31-28-46:536:536 [0] NCCL INFO Plugin Path : /opt/hpcx/nccl_rdma_sharp_plugin/lib/libnccl-net.so
ip-172-31-28-46:536:536 [0] NCCL INFO P2P plugin IBext
ip-172-31-28-46:536:536 [0] NCCL INFO NET/IB : No device found.
ip-172-31-28-46:536:536 [0] NCCL INFO NET/IB : No device found.
ip-172-31-28-46:536:536 [0] NCCL INFO NET/Socket : Using [0]ens5:172.31.28.46<0>
ip-172-31-28-46:536:536 [0] NCCL INFO Using network Socket
ip-172-31-28-46:536:654 [0] NCCL INFO Trees [0] -1/-1/-1->1->0 [1] 0/-1/-1->1->-1
ip-172-31-28-46:536:654 [0] NCCL INFO Channel 00 : 0[1e0] -> 1[1e0] [receive] via NET/Socket/0
ip-172-31-28-46:536:654 [0] NCCL INFO NET/Socket: Using 2 threads and 8 sockets per thread
ip-172-31-28-46:536:654 [0] NCCL INFO Channel 01 : 0[1e0] -> 1[1e0] [receive] via NET/Socket/0
ip-172-31-28-46:536:654 [0] NCCL INFO NET/Socket: Using 2 threads and 8 sockets per thread
ip-172-31-28-46:536:654 [0] NCCL INFO Chan

In [41]:
criterion = nn.CrossEntropyLoss().cuda(GPU)

In [42]:
optimizer = torch.optim.SGD(model.parameters(), LR,
                                momentum=MOMENTUM,
                                weight_decay=WEIGHT_DECAY)

In [43]:
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

In [44]:
transform_train = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.RandomCrop(IMG_SIZE, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_mean_RGB, imagenet_std_RGB),
])

In [45]:
train_dataset = datasets.ImageFolder(
    TRAINDIR, transform=transform_train)

In [46]:
transform_val = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.RandomCrop(IMG_SIZE, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_mean_RGB, imagenet_std_RGB),
])

In [47]:
val_dataset = datasets.ImageFolder(
    VALDIR, transform=transform_val)

In [48]:
train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=TRAIN_BATCH, shuffle=False,
        num_workers=WORKERS, pin_memory=True, sampler=torch.utils.data.distributed.DistributedSampler(train_dataset))

In [49]:
val_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=VAL_BATCH, shuffle=False,
        num_workers=WORKERS, pin_memory=True, sampler=None)

In [50]:
best_acc1 = 0

In [51]:
for epoch in range(START_EPOCH, 2):
#    adjust_learning_rate(optimizer, epoch)

    # train for one epoch
    train(train_loader, model, criterion, optimizer, epoch)

    # evaluate on validation set
    acc1 = validate(val_loader, model, criterion)

    # remember best acc@1 and save checkpoint
    is_best = acc1 > best_acc1
    best_acc1 = max(acc1, best_acc1)


    save_checkpoint({
        'epoch': epoch + 1,
        'arch': ARCH,
        'state_dict': model.state_dict(),
        'best_acc1': best_acc1,
        'optimizer' : optimizer.state_dict(),
    }, is_best)
    
    scheduler.step()
    print('lr: ' + str(scheduler.get_last_lr()[0]))
    
    writer.add_scalar("lr", scheduler.get_last_lr()[0], global_step = global_step)
    
    wandb.log({'lr': scheduler.get_last_lr()[0]})

Epoch: [0][   0/1282]	Time 15.018 (15.018)	Data  3.539 ( 3.539)	Loss 7.0156e+00 (7.0156e+00)	Acc@1   0.00 (  0.00)	Acc@5   0.60 (  0.60)
Epoch: [0][  10/1282]	Time  2.462 ( 2.846)	Data  1.778 ( 1.123)	Loss 6.9121e+00 (6.9762e+00)	Acc@1   0.20 (  0.16)	Acc@5   1.40 (  0.67)
Epoch: [0][  20/1282]	Time  2.336 ( 2.365)	Data  1.672 ( 1.146)	Loss 6.9164e+00 (6.9576e+00)	Acc@1   0.60 (  0.22)	Acc@5   1.40 (  0.90)
Epoch: [0][  30/1282]	Time  3.856 ( 2.249)	Data  3.153 ( 1.208)	Loss 6.8459e+00 (6.9316e+00)	Acc@1   0.00 (  0.25)	Acc@5   1.00 (  0.99)
Epoch: [0][  40/1282]	Time  1.983 ( 2.187)	Data  1.322 ( 1.236)	Loss 6.7243e+00 (6.8868e+00)	Acc@1   1.00 (  0.38)	Acc@5   3.00 (  1.40)
Epoch: [0][  50/1282]	Time  2.560 ( 2.196)	Data  1.849 ( 1.298)	Loss 6.6617e+00 (6.8431e+00)	Acc@1   0.60 (  0.47)	Acc@5   1.80 (  1.69)
Epoch: [0][  60/1282]	Time  2.812 ( 2.132)	Data  2.072 ( 1.268)	Loss 6.4969e+00 (6.7940e+00)	Acc@1   0.80 (  0.55)	Acc@5   4.40 (  1.98)
Epoch: [0][  70/1282]	Time  3.270 ( 2.089

In [52]:
writer.close()
%load_ext tensorboard
%tensorboard --logdir=runs